In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#importing the necessary packages
! pip install -q -U tensorflow-addons
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import pathlib
import keras 
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, SeparableConv2D
from keras.regularizers import l2
from keras.optimizers import SGD, RMSprop
from keras.utils import to_categorical
from keras.layers.normalization import BatchNormalization
from keras.utils.vis_utils import plot_model
from keras.layers import Input, GlobalAveragePooling2D
from keras import models
from keras.models import Model
import tensorflow_addons as tfa

In [ ]:
print("Preparing categories")
DATADIR = "/content/drive/My Drive/Proyecto IA/DataSet"
IMG_SIZE = 224
training_data = []

Preparing categories


In [ ]:
data_dir = pathlib.Path(DATADIR)
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

5580


In [ ]:
print("loading data")
bedroom = list(data_dir.glob('bedroom/*'))
building_outside = list(data_dir.glob('building_outside/*'))
dense_vegetation = list(data_dir.glob('dense_vegetation/*'))
highways = list(data_dir.glob('highways/*'))
livingroom = list(data_dir.glob('livingroom/*'))
skyline = list(data_dir.glob('skyline/*'))
Test_data = list(data_dir.glob('Test_data/*'))
water_bodies = list(data_dir.glob('water_bodies/*'))

loading data


In [ ]:
#creating parameters
batch_size = 32
IMG_SIZE = 224

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(data_dir, validation_split = 0.2, subset = "training", seed = 123, image_size = (IMG_SIZE,IMG_SIZE), batch_size = batch_size)

Found 5811 files belonging to 7 classes.
Using 4649 files for training.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(IMG_SIZE, IMG_SIZE),
  batch_size=batch_size)

Found 5811 files belonging to 7 classes.
Using 1162 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)

['bedroom', 'building_outside', 'dense_vegetation', 'highways', 'livingroom', 'skyline', 'water_bodies']


In [ ]:
#code to see a sample of the data
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
from tensorflow.keras import layers

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Trainning the model begins

In [ ]:
num_classes = 14

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=5
)

Epoch 1/5
146/146 [==============================] - 2147s 15s/step - loss: 1.7803 - accuracy: 0.3048 - val_loss: 1.2338 - val_accuracy: 0.5491
Epoch 2/5
146/146 [==============================] - 272s 2s/step - loss: 1.1002 - accuracy: 0.5910 - val_loss: 1.0743 - val_accuracy: 0.6102
Epoch 3/5
146/146 [==============================] - 252s 2s/step - loss: 0.8258 - accuracy: 0.7080 - val_loss: 1.0510 - val_accuracy: 0.6231
Epoch 4/5
146/146 [==============================] - 252s 2s/step - loss: 0.6279 - accuracy: 0.7799 - val_loss: 1.2833 - val_accuracy: 0.5878
Epoch 5/5
146/146 [==============================] - 252s 2s/step - loss: 0.4765 - accuracy: 0.8316 - val_loss: 1.2986 - val_accuracy: 0.6274


In [ ]:
#!mkdir -p saved_model
model.save('/content/drive/My Drive/Proyecto IA/Model/saved_model/my_model') 

INFO:tensorflow:Assets written to: /content/drive/My Drive/Proyecto IA/Model/saved_model/my_model/assets
